# **Bayesian systems**

<div align = "justify">

**Bayesian systems** rely on Bayesian probability to model uncertainty. They use prior knowledge and update beliefs as new data arrives.

</div>

<div align = "justify">

### **Exercise 1: Emotion detection in text**

System that detects and classifies emotions in text messages using Bayesian techniques.

This can be useful for applications such as sentiment analysis in social media or customer support.

</div>

In [1]:
# Upload data (with percentages)
emotions_prob = {
    'happy': 0.4,
    'sad': 0.3,
    'angry': 0.3
}

# Conditional probabilities
# For example, the first value: P(joyful | happy) = 0.25
words_prob = {
    'happy': {'happy': 0.25, 'joyful': 0.125, 'great': 0.125, 'sad': 0.05},
    'sad': {'happy': 0.05, 'joyful': 0.02, 'great': 0.03, 'sad': 0.20},
    'angry': {'happy': 0.10, 'joyful': 0.03, 'great': 0.02, 'sad': 0.05}
}

In [2]:
def predict_emotion(text):
  words = text.lower().split()  # Preprocessing: lowercasing and tokenizing

  emotions = ['happy', 'sad', 'angry']
  emotion_scores = {}
  total_prob = 0  # This will store the total P(W) for all emotions

  # Check if there are any valid words in the text that exist in the probability tables
  valid_words = [word for word in words if any(word in words_prob[emotion] for emotion in emotions)]

  if not valid_words: # If indeed this is true, the not returns a true. So initiates the if
    return "There are not any of these words in the database."

  # Calculate the total probability P(W) for the word list
  for word in words:
    word_prob = 0

    for emotion in emotions:
        if word in words_prob[emotion]:
            word_prob += emotions_prob[emotion] * words_prob[emotion].get(word, 1e-6)  # Small probability for unknown words
    total_prob += word_prob

  # Calculate the posterior probability for each emotion
  for emotion in emotions:
    sing_prob = emotions_prob[emotion]  # P(E)
    conditional_prob = 1  # Multiply all the conditional probabilities of words given the emotion

    for word in words:
        if word in words_prob[emotion]:
            conditional_prob *= words_prob[emotion].get(word, 1e-6)  # P(W|E) for each word

    posterior_prob = (sing_prob * conditional_prob) / total_prob  # Apply Bayes' Theorem to compute P(E|W)
    emotion_scores[emotion] = posterior_prob

    # Display probabilities for the user
    print(f"Emotion: {emotion}")
    print(f"P(E): {sing_prob}")
    print(f"Conditional probabilities P(W|E):")
    for word in words:
        print(f"  {word}: {words_prob[emotion].get(word, 1e-6)}")
    print(f"Total P(W): {total_prob}")
    print(f"P(E|W): {round(posterior_prob, 6)}")
    print("-" * 40)

  # Return the emotion with the highest score
  return max(emotion_scores, key=emotion_scores.get)

In [8]:
# Demonstration
print("Please, type a sentence:")
example_sentence = input()  # This allows the user to input a sentence manually

# i.e., sentence: Today it is kinda a sad day. Nevertheless I feel great and joyful.

print(f"SENTENCE: {example_sentence}\n\n") # Print the sentence
predicted_emotion = predict_emotion(example_sentence)
print(f"The predicted emotion is: {predicted_emotion}")

Please, type a sentence:
SENTENCE: Today it is kinda a sad day. Nevertheless I feel great and joyful.


Emotion: happy
P(E): 0.4
Conditional probabilities P(W|E):
  today: 1e-06
  it: 1e-06
  is: 1e-06
  kinda: 1e-06
  a: 1e-06
  sad: 0.05
  day.: 1e-06
  nevertheless: 1e-06
  i: 1e-06
  feel: 1e-06
  great: 0.125
  and: 1e-06
  joyful.: 1e-06
Total P(W): 0.16
P(E|W): 0.015625
----------------------------------------
Emotion: sad
P(E): 0.3
Conditional probabilities P(W|E):
  today: 1e-06
  it: 1e-06
  is: 1e-06
  kinda: 1e-06
  a: 1e-06
  sad: 0.2
  day.: 1e-06
  nevertheless: 1e-06
  i: 1e-06
  feel: 1e-06
  great: 0.03
  and: 1e-06
  joyful.: 1e-06
Total P(W): 0.16
P(E|W): 0.01125
----------------------------------------
Emotion: angry
P(E): 0.3
Conditional probabilities P(W|E):
  today: 1e-06
  it: 1e-06
  is: 1e-06
  kinda: 1e-06
  a: 1e-06
  sad: 0.05
  day.: 1e-06
  nevertheless: 1e-06
  i: 1e-06
  feel: 1e-06
  great: 0.02
  and: 1e-06
  joyful.: 1e-06
Total P(W): 0.16
P(E|W): 0

<div align = "justify">

With the sentence "Today it is kinda a sad day. Nevertheless I feel great and joyful.", the system would predict the person who typed this is 40% happy, 30% sad and 30% angry. Thus, the predicted emotion is **happy**.

</div>

<div align = "justify">

### **Exercise 2:** Driver Assistance System using Bayesian Networks and Expert Rules

* *Objective:* Understand the application of Bayesian networks in driver assistance systems, combined with an expert system for rule-based reasoning.

* *Scenario:* We are designing a driver assistance system that predicts the likelihood of a collision based on the speed of the vehicle, the distance to the vehicle ahead, the condition of the road (wet or dry), and additional expert rules.
</div>

In [10]:
# !pip install pgmpy # Install library for Probabilistic graphical models (This is how it's downloaded in google colab)

from pgmpy.models import DiscreteBayesianNetwork 
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

In [14]:
# Define the structure
model = DiscreteBayesianNetwork ([('Speed', 'CollisionRisk'),
                         ('Distance', 'CollisionRisk'),
                         ('RoadCondition', 'CollisionRisk')])

# Define the CPTs (Conditional Probability Tables)

# P(Speed) = 'Low' (0) or 'High' (1)
cpd_speed = TabularCPD(variable='Speed', variable_card=2, values=[[0.6], [0.4]])

# P(Distance) = 'Short' (0) or 'Long' (1)
cpd_distance = TabularCPD(variable='Distance', variable_card=2, values=[[0.6], [0.4]])

# P(RoadCondition) = 'Dry' (0) or 'Wet' (1)
cpd_road_condition = TabularCPD(variable='RoadCondition', variable_card=2, values=[[0.8], [0.2]])

# Values represent the probability of a collision for all different combinations of Speed, Distance, and RoadCondition
cpd_collision_risk = TabularCPD(variable='CollisionRisk', variable_card=2,
                                  values=[[0.95, 0.7, 0.85, 0.4, 0.99, 0.7, 0.74, 0.2],
                                          [0.05, 0.3, 0.15, 0.6, 0.01, 0.3, 0.26, 0.8]],

                                # P(CollisionRisk|Speed=Low, Distance=Short) = 0.95 ...
                                  evidence=['Speed', 'Distance', 'RoadCondition'],
                                  evidence_card=[2, 2, 2])

# Add the CPTs to the model
model.add_cpds(cpd_speed, cpd_distance, cpd_road_condition, cpd_collision_risk)

# Check if the model is valid
assert model.check_model()

# Perform inference
inference = VariableElimination(model)

# Simulate a scenario: High speed, short distance, wet road
result = inference.query(variables=['CollisionRisk'],
                         evidence={'Speed': 1, 'Distance': 0, 'RoadCondition': 1})
print(result)

+------------------+----------------------+
| CollisionRisk    |   phi(CollisionRisk) |
+==================+======================+
| CollisionRisk(0) |               0.7000 |
+------------------+----------------------+
| CollisionRisk(1) |               0.3000 |
+------------------+----------------------+


<div align = "justify">

Once a scenario is simulated, where the driver travels a short distance, at a high speed and the road is wet, then the Bayesian system would predict the collision risk is no with 70% and yes with 30%.

</div>